<a href="https://colab.research.google.com/github/b-paramesh/Crypto-Portfolio-Manager/blob/main/Crypto_Portfolio_Manager_Sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✔ Create Database
✔ Create Tables
✔ Constraints
✔ Primary & Foreign Keys
✔ Insert / Update / Delete
✔ DISTINCT
✔ ORDER BY
✔ LIMIT
✔ INNER JOIN
✔ LEFT JOIN
✔ GROUP BY
✔ HAVING
✔ Aggregate Functions
✔ Subqueries
✔ EXISTS
✔ CASE
✔ Index
✔ View
✔ Trigger
✔ Transaction
✔ Drop Table
✔ Close Connection

In [1]:
# =====================================================
# COMPLETE SQL FEATURES DEMO (SQLite in Colab)
# =====================================================

import sqlite3

print("==========================================")
print("   COMPLETE SQL FEATURES DEMO (SQLite)")
print("==========================================")

# 1️⃣ Connect to Database
conn = sqlite3.connect("advanced_sql_demo.db")
cursor = conn.cursor()

cursor.execute("PRAGMA foreign_keys = ON;")
print("Database Connected!")

# =====================================================
# 2️⃣ Create Tables with Constraints
# =====================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    age INTEGER CHECK(age >= 18),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS jobs (
    job_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    salary INTEGER CHECK(salary > 0),
    location TEXT DEFAULT 'Remote'
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS applications (
    application_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    job_id INTEGER,
    status TEXT DEFAULT 'Applied',
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (job_id) REFERENCES jobs(job_id)
);
""")

conn.commit()
print("Tables Created!")

# =====================================================
# 3️⃣ Insert Data
# =====================================================

users_data = [
    ("Rahul", "rahul@gmail.com", 25),
    ("Anjali", "anjali@gmail.com", 30),
    ("Kiran", "kiran@gmail.com", 22)
]

cursor.executemany("INSERT INTO users (name, email, age) VALUES (?, ?, ?)", users_data)

jobs_data = [
    ("Backend Developer", 800000, "Hyderabad"),
    ("Frontend Developer", 700000, "Bangalore"),
    ("Data Analyst", 600000, "Remote")
]

cursor.executemany("INSERT INTO jobs (title, salary, location) VALUES (?, ?, ?)", jobs_data)

applications_data = [
    (1, 1),
    (2, 2),
    (1, 3)
]

cursor.executemany("INSERT INTO applications (user_id, job_id) VALUES (?, ?)", applications_data)

conn.commit()
print("Data Inserted!")

# =====================================================
# 4️⃣ SELECT + DISTINCT + ORDER BY + LIMIT
# =====================================================

print("\nDistinct Locations:")
cursor.execute("SELECT DISTINCT location FROM jobs ORDER BY location DESC LIMIT 3;")
print(cursor.fetchall())

# =====================================================
# 5️⃣ JOIN (INNER + LEFT)
# =====================================================

print("\nINNER JOIN:")
cursor.execute("""
SELECT u.name, j.title
FROM applications a
INNER JOIN users u ON a.user_id = u.user_id
INNER JOIN jobs j ON a.job_id = j.job_id;
""")
print(cursor.fetchall())

print("\nLEFT JOIN:")
cursor.execute("""
SELECT u.name, j.title
FROM users u
LEFT JOIN applications a ON u.user_id = a.user_id
LEFT JOIN jobs j ON a.job_id = j.job_id;
""")
print(cursor.fetchall())

# =====================================================
# 6️⃣ GROUP BY + HAVING
# =====================================================

print("\nApplications per User:")
cursor.execute("""
SELECT user_id, COUNT(*) as total
FROM applications
GROUP BY user_id
HAVING total >= 1;
""")
print(cursor.fetchall())

# =====================================================
# 7️⃣ Aggregate Functions
# =====================================================

print("\nAverage Salary:")
cursor.execute("SELECT AVG(salary) FROM jobs;")
print(cursor.fetchone())

# =====================================================
# 8️⃣ Subquery + EXISTS
# =====================================================

print("\nUsers Applied to Job 1:")
cursor.execute("""
SELECT name FROM users
WHERE EXISTS (
    SELECT 1 FROM applications
    WHERE applications.user_id = users.user_id
    AND job_id = 1
);
""")
print(cursor.fetchall())

# =====================================================
# 9️⃣ CASE Statement
# =====================================================

print("\nSalary Category:")
cursor.execute("""
SELECT title,
CASE
    WHEN salary >= 800000 THEN 'High'
    WHEN salary >= 600000 THEN 'Medium'
    ELSE 'Low'
END as category
FROM jobs;
""")
print(cursor.fetchall())

# =====================================================
# 🔟 Update & Delete
# =====================================================

cursor.execute("UPDATE users SET age = 26 WHERE name = 'Rahul';")
cursor.execute("DELETE FROM users WHERE name = 'Kiran';")
conn.commit()

print("\nUpdated Users:")
cursor.execute("SELECT * FROM users;")
print(cursor.fetchall())

# =====================================================
# 1️⃣1️⃣ Index
# =====================================================

cursor.execute("CREATE INDEX IF NOT EXISTS idx_email ON users(email);")
conn.commit()
print("\nIndex Created on email")

# =====================================================
# 1️⃣2️⃣ View
# =====================================================

cursor.execute("""
CREATE VIEW IF NOT EXISTS user_job_view AS
SELECT u.name, j.title
FROM applications a
JOIN users u ON a.user_id = u.user_id
JOIN jobs j ON a.job_id = j.job_id;
""")

print("\nView Result:")
cursor.execute("SELECT * FROM user_job_view;")
print(cursor.fetchall())

# =====================================================
# 1️⃣3️⃣ Trigger
# =====================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS logs (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    message TEXT
);
""")

cursor.execute("""
CREATE TRIGGER IF NOT EXISTS log_user_insert
AFTER INSERT ON users
BEGIN
    INSERT INTO logs(message) VALUES ('New user added');
END;
""")

cursor.execute("INSERT INTO users (name, email, age) VALUES (?, ?, ?)",
               ("TestUser", "test@gmail.com", 28))
conn.commit()

print("\nTrigger Logs:")
cursor.execute("SELECT * FROM logs;")
print(cursor.fetchall())

# =====================================================
# 1️⃣4️⃣ Transaction Example
# =====================================================

try:
    conn.execute("BEGIN")
    cursor.execute("INSERT INTO users (name, email, age) VALUES (?, ?, ?)",
                   ("TempUser", "temp@gmail.com", 24))
    conn.commit()
    print("\nTransaction Successful!")
except:
    conn.rollback()
    print("Transaction Failed!")

# =====================================================
# 1️⃣5️⃣ Drop Table Example (Optional)
# =====================================================

# cursor.execute("DROP TABLE logs;")

# =====================================================
# Close Connection
# =====================================================

conn.close()
print("\nDatabase Closed Successfully!")

print("\n==========================================")
print("        END OF COMPLETE SQL DEMO")
print("==========================================")

   COMPLETE SQL FEATURES DEMO (SQLite)
Database Connected!
Tables Created!
Data Inserted!

Distinct Locations:
[('Remote',), ('Hyderabad',), ('Bangalore',)]

INNER JOIN:
[('Rahul', 'Backend Developer'), ('Anjali', 'Frontend Developer'), ('Rahul', 'Data Analyst')]

LEFT JOIN:
[('Rahul', 'Backend Developer'), ('Rahul', 'Data Analyst'), ('Anjali', 'Frontend Developer'), ('Kiran', None)]

Applications per User:
[(1, 2), (2, 1)]

Average Salary:
(700000.0,)

Users Applied to Job 1:
[('Rahul',)]

Salary Category:
[('Backend Developer', 'High'), ('Frontend Developer', 'Medium'), ('Data Analyst', 'Medium')]

Updated Users:
[(1, 'Rahul', 'rahul@gmail.com', 26, '2026-02-20 12:47:42'), (2, 'Anjali', 'anjali@gmail.com', 30, '2026-02-20 12:47:42')]

Index Created on email

View Result:
[('Rahul', 'Backend Developer'), ('Anjali', 'Frontend Developer'), ('Rahul', 'Data Analyst')]

Trigger Logs:
[(1, 'New user added')]

Transaction Successful!

Database Closed Successfully!

        END OF COMPLETE SQ

✔ 3NF Normalized Structure
✔ Lookup Tables
✔ Many-to-Many Relationship
✔ Composite Primary Key
✔ Data Integrity Constraints
✔ CHECK Constraints
✔ Default Values
✔ Soft Delete Flag
✔ Audit Logging
✔ Cascading Rules
✔ Index Optimization
✔ Naming Conventions
✔ Clean Architecture DB Design

In [2]:
# ==========================================================
# PROPER TABLE DESIGN - PRODUCTION LEVEL (SQLite in Colab)
# ==========================================================

import sqlite3

print("==============================================")
print("   PROPER TABLE DESIGN - PRODUCTION DEMO")
print("==============================================")

conn = sqlite3.connect("proper_design.db")
cursor = conn.cursor()

cursor.execute("PRAGMA foreign_keys = ON;")

# ==========================================================
# 1️⃣ LOOKUP TABLES (Reference Tables)
# ==========================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS roles (
    role_id INTEGER PRIMARY KEY AUTOINCREMENT,
    role_name TEXT UNIQUE NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS job_types (
    job_type_id INTEGER PRIMARY KEY AUTOINCREMENT,
    type_name TEXT UNIQUE NOT NULL
);
""")

# ==========================================================
# 2️⃣ USERS TABLE (3NF Design)
# ==========================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    phone TEXT UNIQUE,
    role_id INTEGER NOT NULL,
    is_active INTEGER DEFAULT 1 CHECK (is_active IN (0,1)),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,

    FOREIGN KEY (role_id) REFERENCES roles(role_id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);
""")

# ==========================================================
# 3️⃣ JOBS TABLE
# ==========================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS jobs (
    job_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT NOT NULL,
    salary INTEGER CHECK (salary > 0),
    job_type_id INTEGER NOT NULL,
    location TEXT NOT NULL,
    is_active INTEGER DEFAULT 1 CHECK (is_active IN (0,1)),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,

    FOREIGN KEY (job_type_id) REFERENCES job_types(job_type_id)
        ON DELETE RESTRICT
);
""")

# ==========================================================
# 4️⃣ MANY-TO-MANY TABLE (Applications)
# ==========================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS applications (
    user_id INTEGER,
    job_id INTEGER,
    status TEXT DEFAULT 'Applied',
    applied_at DATETIME DEFAULT CURRENT_TIMESTAMP,

    PRIMARY KEY (user_id, job_id),

    FOREIGN KEY (user_id) REFERENCES users(user_id)
        ON DELETE CASCADE,
    FOREIGN KEY (job_id) REFERENCES jobs(job_id)
        ON DELETE CASCADE
);
""")

# ==========================================================
# 5️⃣ SKILLS TABLE
# ==========================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS skills (
    skill_id INTEGER PRIMARY KEY AUTOINCREMENT,
    skill_name TEXT UNIQUE NOT NULL
);
""")

# ==========================================================
# 6️⃣ MANY-TO-MANY USER_SKILLS
# ==========================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS user_skills (
    user_id INTEGER,
    skill_id INTEGER,

    PRIMARY KEY (user_id, skill_id),

    FOREIGN KEY (user_id) REFERENCES users(user_id)
        ON DELETE CASCADE,
    FOREIGN KEY (skill_id) REFERENCES skills(skill_id)
        ON DELETE CASCADE
);
""")

# ==========================================================
# 7️⃣ INDEXES FOR PERFORMANCE
# ==========================================================

cursor.execute("CREATE INDEX IF NOT EXISTS idx_users_email ON users(email);")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_jobs_salary ON jobs(salary);")

# ==========================================================
# 8️⃣ AUDIT LOG TABLE
# ==========================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS audit_logs (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    table_name TEXT NOT NULL,
    operation TEXT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

# ==========================================================
# 9️⃣ TRIGGER FOR AUDIT
# ==========================================================

cursor.execute("""
CREATE TRIGGER IF NOT EXISTS user_insert_audit
AFTER INSERT ON users
BEGIN
    INSERT INTO audit_logs (table_name, operation)
    VALUES ('users', 'INSERT');
END;
""")

conn.commit()

print("All Tables Created Successfully!")

# ==========================================================
# 🔟 INSERT SAMPLE LOOKUP DATA
# ==========================================================

cursor.executemany("INSERT OR IGNORE INTO roles(role_name) VALUES (?)",
                   [("Admin",), ("Recruiter",), ("Candidate",)])

cursor.executemany("INSERT OR IGNORE INTO job_types(type_name) VALUES (?)",
                   [("Full-Time",), ("Part-Time",), ("Internship",)])

conn.commit()

print("Lookup Data Inserted!")

# ==========================================================
# 1️⃣1️⃣ INSERT SAMPLE USER & JOB
# ==========================================================

cursor.execute("""
INSERT INTO users(first_name, last_name, email, role_id)
VALUES ('Rahul', 'Kumar', 'rahul@gmail.com', 3);
""")

cursor.execute("""
INSERT INTO jobs(title, description, salary, job_type_id, location)
VALUES ('Backend Developer', 'API Development', 800000, 1, 'Hyderabad');
""")

conn.commit()

print("Sample Data Inserted!")

# ==========================================================
# 1️⃣2️⃣ VERIFY DESIGN
# ==========================================================

print("\nUsers:")
cursor.execute("SELECT * FROM users;")
print(cursor.fetchall())

print("\nJobs:")
cursor.execute("SELECT * FROM jobs;")
print(cursor.fetchall())

print("\nAudit Logs:")
cursor.execute("SELECT * FROM audit_logs;")
print(cursor.fetchall())

# ==========================================================
# CLOSE
# ==========================================================

conn.close()
print("\nDatabase Closed Successfully!")

print("\n==============================================")
print("        END OF PROPER TABLE DESIGN")
print("==============================================")

   PROPER TABLE DESIGN - PRODUCTION DEMO
All Tables Created Successfully!
Lookup Data Inserted!
Sample Data Inserted!

Users:
[(1, 'Rahul', 'Kumar', 'rahul@gmail.com', None, 3, 1, '2026-02-20 12:51:11', '2026-02-20 12:51:11')]

Jobs:
[(1, 'Backend Developer', 'API Development', 800000, 1, 'Hyderabad', 1, '2026-02-20 12:51:11')]

Audit Logs:
[(1, 'users', 'INSERT', '2026-02-20 12:51:11')]

Database Closed Successfully!

        END OF PROPER TABLE DESIGN


✔ PRIMARY KEY

Unique identifier

Cannot be NULL

✔ AUTOINCREMENT

Auto-generates ID

✔ UNIQUE

No duplicate values allowed

✔ NOT NULL

Field must contain value

✔ CHECK

Validates condition (salary > 0)

✔ DEFAULT

Auto assigns value if not provided

✔ FOREIGN KEY

Maintains referential integrity

✔ COMPOSITE PRIMARY KEY

Multiple columns as primary key

✔ CASCADE DELETE

Deletes related records automatically

✔ INDEX

Improves query performance

In [3]:
# =====================================================
# COMPLETE SQL CONSTRAINTS DEMO (SQLite - Colab Ready)
# =====================================================

import sqlite3

print("==============================================")
print("        SQL CONSTRAINTS COMPLETE DEMO")
print("==============================================")

conn = sqlite3.connect("constraints_demo.db")
cursor = conn.cursor()

# Enable Foreign Key Support
cursor.execute("PRAGMA foreign_keys = ON;")

# =====================================================
# 1️⃣ PRIMARY KEY + AUTOINCREMENT
# =====================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE,
    salary INTEGER CHECK (salary > 0),
    department TEXT DEFAULT 'General'
);
""")

print("Table employees created with:")
print("PRIMARY KEY, AUTOINCREMENT, NOT NULL, UNIQUE, CHECK, DEFAULT")

# =====================================================
# 2️⃣ FOREIGN KEY CONSTRAINT
# =====================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS departments (
    dept_id INTEGER PRIMARY KEY AUTOINCREMENT,
    dept_name TEXT UNIQUE NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS emp_department (
    emp_id INTEGER,
    dept_id INTEGER,
    PRIMARY KEY (emp_id, dept_id),
    FOREIGN KEY (emp_id) REFERENCES employees(emp_id)
        ON DELETE CASCADE,
    FOREIGN KEY (dept_id) REFERENCES departments(dept_id)
        ON DELETE CASCADE
);
""")

print("Foreign Key & Composite Primary Key Created!")

# =====================================================
# 3️⃣ INSERT VALID DATA
# =====================================================

cursor.execute("INSERT INTO departments (dept_name) VALUES ('IT')")
cursor.execute("INSERT INTO employees (name, email, salary) VALUES (?, ?, ?)",
               ("Rahul", "rahul@gmail.com", 50000))

cursor.execute("INSERT INTO emp_department (emp_id, dept_id) VALUES (1, 1)")

conn.commit()
print("Valid Data Inserted Successfully!")

# =====================================================
# 4️⃣ UNIQUE CONSTRAINT TEST
# =====================================================

try:
    cursor.execute("INSERT INTO employees (name, email, salary) VALUES (?, ?, ?)",
                   ("Another Rahul", "rahul@gmail.com", 60000))
    conn.commit()
except Exception as e:
    print("\nUNIQUE Constraint Error:", e)

# =====================================================
# 5️⃣ CHECK CONSTRAINT TEST
# =====================================================

try:
    cursor.execute("INSERT INTO employees (name, email, salary) VALUES (?, ?, ?)",
                   ("Invalid Salary", "invalid@gmail.com", -100))
    conn.commit()
except Exception as e:
    print("\nCHECK Constraint Error:", e)

# =====================================================
# 6️⃣ NOT NULL CONSTRAINT TEST
# =====================================================

try:
    cursor.execute("INSERT INTO employees (name, email, salary) VALUES (?, ?, ?)",
                   (None, "null@gmail.com", 40000))
    conn.commit()
except Exception as e:
    print("\nNOT NULL Constraint Error:", e)

# =====================================================
# 7️⃣ DEFAULT CONSTRAINT
# =====================================================

cursor.execute("INSERT INTO employees (name, email, salary) VALUES (?, ?, ?)",
               ("Anjali", "anjali@gmail.com", 60000))
conn.commit()

print("\nDEFAULT Value Example:")
cursor.execute("SELECT name, department FROM employees WHERE name='Anjali'")
print(cursor.fetchall())

# =====================================================
# 8️⃣ CASCADE DELETE TEST
# =====================================================

cursor.execute("DELETE FROM employees WHERE emp_id = 1")
conn.commit()

print("\nAfter DELETE CASCADE:")
cursor.execute("SELECT * FROM emp_department")
print(cursor.fetchall())

# =====================================================
# 9️⃣ INDEX (Related to UNIQUE & Performance)
# =====================================================

cursor.execute("CREATE INDEX IF NOT EXISTS idx_salary ON employees(salary)")
conn.commit()
print("\nIndex Created on salary")

# =====================================================
# 🔟 SHOW FINAL DATA
# =====================================================

print("\nFinal Employees Table:")
cursor.execute("SELECT * FROM employees")
print(cursor.fetchall())

# =====================================================
# CLOSE
# =====================================================

conn.close()

print("\n==============================================")
print("        END OF CONSTRAINTS DEMO")
print("==============================================")

        SQL CONSTRAINTS COMPLETE DEMO
Table employees created with:
PRIMARY KEY, AUTOINCREMENT, NOT NULL, UNIQUE, CHECK, DEFAULT
Foreign Key & Composite Primary Key Created!
Valid Data Inserted Successfully!

UNIQUE Constraint Error: UNIQUE constraint failed: employees.email

CHECK Constraint Error: CHECK constraint failed: salary > 0

NOT NULL Constraint Error: NOT NULL constraint failed: employees.name

DEFAULT Value Example:
[('Anjali', 'General')]

After DELETE CASCADE:
[]

Index Created on salary

Final Employees Table:
[(2, 'Anjali', 'anjali@gmail.com', 60000, 'General')]

        END OF CONSTRAINTS DEMO


✔ One-to-One

UNIQUE foreign key

✔ One-to-Many

Foreign key in child table

✔ Many-to-Many

Junction table

Composite primary key

✔ Self-Referencing

Table referencing itself

✔ CASCADE DELETE

Automatically removes related rows

✔ SET NULL

Keeps child but removes reference

In [4]:
# =====================================================
# COMPLETE SQL RELATIONSHIPS DEMO (SQLite - Colab Ready)
# =====================================================

import sqlite3

print("==============================================")
print("        SQL RELATIONSHIPS COMPLETE DEMO")
print("==============================================")

conn = sqlite3.connect("relationships_demo.db")
cursor = conn.cursor()

cursor.execute("PRAGMA foreign_keys = ON;")

# =====================================================
# 1️⃣ ONE-TO-ONE RELATIONSHIP
# Example: User ↔ User_Profile
# =====================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS user_profiles (
    profile_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER UNIQUE,  -- UNIQUE ensures one-to-one
    bio TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
        ON DELETE CASCADE
);
""")

cursor.execute("INSERT INTO users (name) VALUES ('Rahul')")
cursor.execute("INSERT INTO user_profiles (user_id, bio) VALUES (1, 'Software Developer')")
conn.commit()

print("\nOne-to-One Example:")
cursor.execute("""
SELECT u.name, p.bio
FROM users u
JOIN user_profiles p ON u.user_id = p.user_id
""")
print(cursor.fetchall())

# =====================================================
# 2️⃣ ONE-TO-MANY RELATIONSHIP
# Example: Department → Employees
# =====================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS departments (
    dept_id INTEGER PRIMARY KEY AUTOINCREMENT,
    dept_name TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    dept_id INTEGER,
    FOREIGN KEY (dept_id) REFERENCES departments(dept_id)
        ON DELETE SET NULL
);
""")

cursor.execute("INSERT INTO departments (dept_name) VALUES ('IT')")
cursor.execute("INSERT INTO employees (name, dept_id) VALUES ('Anjali', 1)")
cursor.execute("INSERT INTO employees (name, dept_id) VALUES ('Kiran', 1)")
conn.commit()

print("\nOne-to-Many Example:")
cursor.execute("""
SELECT d.dept_name, e.name
FROM departments d
JOIN employees e ON d.dept_id = e.dept_id
""")
print(cursor.fetchall())

# =====================================================
# 3️⃣ MANY-TO-MANY RELATIONSHIP
# Example: Students ↔ Courses
# =====================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_name TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS courses (
    course_id INTEGER PRIMARY KEY AUTOINCREMENT,
    course_name TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS enrollments (
    student_id INTEGER,
    course_id INTEGER,
    PRIMARY KEY (student_id, course_id),
    FOREIGN KEY (student_id) REFERENCES students(student_id)
        ON DELETE CASCADE,
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
        ON DELETE CASCADE
);
""")

cursor.execute("INSERT INTO students (student_name) VALUES ('Rahul')")
cursor.execute("INSERT INTO students (student_name) VALUES ('Anjali')")
cursor.execute("INSERT INTO courses (course_name) VALUES ('Python')")
cursor.execute("INSERT INTO courses (course_name) VALUES ('SQL')")

cursor.execute("INSERT INTO enrollments VALUES (1, 1)")
cursor.execute("INSERT INTO enrollments VALUES (1, 2)")
cursor.execute("INSERT INTO enrollments VALUES (2, 2)")
conn.commit()

print("\nMany-to-Many Example:")
cursor.execute("""
SELECT s.student_name, c.course_name
FROM enrollments e
JOIN students s ON e.student_id = s.student_id
JOIN courses c ON e.course_id = c.course_id
""")
print(cursor.fetchall())

# =====================================================
# 4️⃣ SELF-REFERENCING RELATIONSHIP
# Example: Employee → Manager
# =====================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS staff (
    staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    manager_id INTEGER,
    FOREIGN KEY (manager_id) REFERENCES staff(staff_id)
        ON DELETE SET NULL
);
""")

cursor.execute("INSERT INTO staff (name) VALUES ('CEO')")
cursor.execute("INSERT INTO staff (name, manager_id) VALUES ('Manager', 1)")
cursor.execute("INSERT INTO staff (name, manager_id) VALUES ('Employee', 2)")
conn.commit()

print("\nSelf-Referencing Example:")
cursor.execute("""
SELECT e.name AS Employee, m.name AS Manager
FROM staff e
LEFT JOIN staff m ON e.manager_id = m.staff_id
""")
print(cursor.fetchall())

# =====================================================
# 5️⃣ CASCADE DELETE TEST
# =====================================================

cursor.execute("DELETE FROM students WHERE student_id = 1")
conn.commit()

print("\nAfter CASCADE DELETE (Many-to-Many):")
cursor.execute("SELECT * FROM enrollments")
print(cursor.fetchall())

# =====================================================
# CLOSE CONNECTION
# =====================================================

conn.close()

print("\n==============================================")
print("        END OF RELATIONSHIPS DEMO")
print("==============================================")

        SQL RELATIONSHIPS COMPLETE DEMO

One-to-One Example:
[('Rahul', 'Software Developer')]

One-to-Many Example:
[('IT', 'Anjali'), ('IT', 'Kiran')]

Many-to-Many Example:
[('Rahul', 'Python'), ('Rahul', 'SQL'), ('Anjali', 'SQL')]

Self-Referencing Example:
[('CEO', None), ('Manager', 'CEO'), ('Employee', 'Manager')]

After CASCADE DELETE (Many-to-Many):
[(2, 2)]

        END OF RELATIONSHIPS DEMO


Query optimization

Index strategy

Composite index logic

Partial indexing

Query plan analysis

Performance tuning basics

In [5]:
"""
=========================================================
            COMPLETE SQL INDEXING MASTER DEMO
=========================================================
This script demonstrates:

✔ What is Index
✔ Why Indexing is needed
✔ Single Column Index
✔ Composite Index
✔ Unique Index
✔ Partial Index (SQLite)
✔ Expression Index
✔ Query Plan Analysis
✔ Drop Index
✔ When NOT to use Index
=========================================================
"""

import sqlite3
import random
import string
import time

print("=================================================")
print("         SQL INDEXING MASTER DEMO")
print("=================================================")

# =================================================
# 1️⃣ Connect to Database
# =================================================

conn = sqlite3.connect("index_master.db")
cursor = conn.cursor()

# =================================================
# 2️⃣ Create Large Table
# =================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL,
    department TEXT,
    salary INTEGER
);
""")

conn.commit()
print("\nTable Created Successfully!")

# =================================================
# 3️⃣ Insert Large Data (Performance Demo)
# =================================================

departments = ["IT", "HR", "Finance", "Marketing"]

print("\nInserting 5000 records...")

for i in range(5000):
    cursor.execute("""
    INSERT INTO users (name, email, department, salary)
    VALUES (?, ?, ?, ?)
    """, (
        f"User{i}",
        f"user{i}@gmail.com",
        random.choice(departments),
        random.randint(30000, 150000)
    ))

conn.commit()
print("Data Inserted!")

# =================================================
# 4️⃣ Query WITHOUT Index
# =================================================

print("\nQuery Plan WITHOUT Index:")

cursor.execute("""
EXPLAIN QUERY PLAN
SELECT * FROM users WHERE email='user4000@gmail.com'
""")

print(cursor.fetchall())

# =================================================
# 5️⃣ Create Single Column Index
# =================================================

cursor.execute("CREATE INDEX idx_users_email ON users(email);")
conn.commit()

print("\nSingle Column Index Created on email!")

print("\nQuery Plan WITH Index:")

cursor.execute("""
EXPLAIN QUERY PLAN
SELECT * FROM users WHERE email='user4000@gmail.com'
""")

print(cursor.fetchall())

# =================================================
# 6️⃣ Composite Index
# =================================================

cursor.execute("""
CREATE INDEX idx_users_dept_salary
ON users(department, salary);
""")

conn.commit()
print("\nComposite Index Created (department, salary)")

cursor.execute("""
EXPLAIN QUERY PLAN
SELECT * FROM users
WHERE department='IT' AND salary > 70000;
""")

print("Composite Index Query Plan:", cursor.fetchall())

# =================================================
# 7️⃣ Unique Index
# =================================================

cursor.execute("""
CREATE UNIQUE INDEX idx_unique_email
ON users(email);
""")

conn.commit()
print("\nUnique Index Created!")

# =================================================
# 8️⃣ Partial Index (SQLite Feature)
# =================================================

cursor.execute("""
CREATE INDEX idx_high_salary
ON users(salary)
WHERE salary > 100000;
""")

conn.commit()
print("\nPartial Index Created (salary > 100000)")

# =================================================
# 9️⃣ Expression Index
# =================================================

cursor.execute("""
CREATE INDEX idx_lower_email
ON users(LOWER(email));
""")

conn.commit()
print("\nExpression Index Created (LOWER(email))")

# =================================================
# 🔟 View Existing Indexes
# =================================================

print("\nList of Indexes:")
cursor.execute("PRAGMA index_list('users');")
print(cursor.fetchall())

# =================================================
# 1️⃣1️⃣ Drop Index
# =================================================

cursor.execute("DROP INDEX IF EXISTS idx_lower_email;")
conn.commit()
print("\nExpression Index Dropped!")

# =================================================
# 1️⃣2️⃣ Performance Comparison (Timing)
# =================================================

def measure_query(query):
    start = time.time()
    cursor.execute(query)
    cursor.fetchall()
    end = time.time()
    return end - start

time_with_index = measure_query(
    "SELECT * FROM users WHERE email='user3000@gmail.com'"
)

print("\nQuery Execution Time With Index:", time_with_index)

# =================================================
# Close Database
# =================================================

conn.close()

print("\n=================================================")
print("        END OF INDEXING MASTER DEMO")
print("=================================================")

         SQL INDEXING MASTER DEMO

Table Created Successfully!

Inserting 5000 records...
Data Inserted!

Query Plan WITHOUT Index:
[(2, 0, 0, 'SCAN users')]

Single Column Index Created on email!

Query Plan WITH Index:
[(3, 0, 0, 'SEARCH users USING INDEX idx_users_email (email=?)')]

Composite Index Created (department, salary)
Composite Index Query Plan: [(3, 0, 0, 'SEARCH users USING INDEX idx_users_dept_salary (department=? AND salary>?)')]

Unique Index Created!

Partial Index Created (salary > 100000)

Expression Index Created (LOWER(email))

List of Indexes:
[(0, 'idx_lower_email', 0, 'c', 0), (1, 'idx_high_salary', 0, 'c', 1), (2, 'idx_unique_email', 1, 'c', 0), (3, 'idx_users_dept_salary', 0, 'c', 0), (4, 'idx_users_email', 0, 'c', 0)]

Expression Index Dropped!

Query Execution Time With Index: 9.632110595703125e-05

        END OF INDEXING MASTER DEMO


✔ Basic Views
✔ Conditional Views
✔ Join Views
✔ Aggregated Views
✔ View Metadata
✔ Dropping Views
✔ Real-world usage

In [6]:
"""
=========================================================
            COMPLETE SQL VIEWS MASTER DEMO
=========================================================
This script demonstrates:

✔ What is a View
✔ Simple View
✔ View with WHERE
✔ View with JOIN
✔ View with Aggregation
✔ Read-only behavior
✔ Updating through View (SQLite limits)
✔ Dropping View
✔ View vs Table difference
✔ Real-world reporting examples
=========================================================
"""

import sqlite3

print("=================================================")
print("           SQL VIEWS MASTER DEMO")
print("=================================================")

# =================================================
# 1️⃣ Connect Database
# =================================================

conn = sqlite3.connect("views_demo.db")
cursor = conn.cursor()

cursor.execute("PRAGMA foreign_keys = ON;")

# =================================================
# 2️⃣ Create Base Tables
# =================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    city TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    amount INTEGER,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
""")

conn.commit()
print("\nTables Created!")

# =================================================
# 3️⃣ Insert Sample Data
# =================================================

cursor.executemany("""
INSERT INTO customers (name, city)
VALUES (?, ?)
""", [
    ("Rahul", "Hyderabad"),
    ("Anjali", "Bangalore"),
    ("Kiran", "Hyderabad")
])

cursor.executemany("""
INSERT INTO orders (customer_id, amount)
VALUES (?, ?)
""", [
    (1, 5000),
    (1, 3000),
    (2, 7000),
    (3, 2000)
])

conn.commit()
print("Sample Data Inserted!")

# =================================================
# 4️⃣ Simple View
# =================================================

cursor.execute("""
CREATE VIEW IF NOT EXISTS view_customers AS
SELECT name, city FROM customers;
""")

print("\nSimple View Result:")
cursor.execute("SELECT * FROM view_customers;")
print(cursor.fetchall())

# =================================================
# 5️⃣ View with WHERE Condition
# =================================================

cursor.execute("""
CREATE VIEW IF NOT EXISTS view_hyderabad_customers AS
SELECT * FROM customers
WHERE city = 'Hyderabad';
""")

print("\nView with WHERE Result:")
cursor.execute("SELECT * FROM view_hyderabad_customers;")
print(cursor.fetchall())

# =================================================
# 6️⃣ View with JOIN
# =================================================

cursor.execute("""
CREATE VIEW IF NOT EXISTS view_customer_orders AS
SELECT c.name, o.amount
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id;
""")

print("\nView with JOIN Result:")
cursor.execute("SELECT * FROM view_customer_orders;")
print(cursor.fetchall())

# =================================================
# 7️⃣ View with Aggregation
# =================================================

cursor.execute("""
CREATE VIEW IF NOT EXISTS view_total_orders AS
SELECT c.name, SUM(o.amount) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.name;
""")

print("\nView with Aggregation Result:")
cursor.execute("SELECT * FROM view_total_orders;")
print(cursor.fetchall())

# =================================================
# 8️⃣ Updating Data Through View (Limited in SQLite)
# =================================================

print("\nUpdating Base Table (View Reflects Change)")

cursor.execute("UPDATE customers SET city='Chennai' WHERE name='Kiran'")
conn.commit()

cursor.execute("SELECT * FROM view_customers;")
print(cursor.fetchall())

# =================================================
# 9️⃣ View Metadata
# =================================================

print("\nList of Views in Database:")
cursor.execute("""
SELECT name FROM sqlite_master
WHERE type='view';
""")
print(cursor.fetchall())

# =================================================
# 🔟 Drop View
# =================================================

cursor.execute("DROP VIEW IF EXISTS view_hyderabad_customers;")
conn.commit()
print("\nDropped view_hyderabad_customers")

# =================================================
# Close Connection
# =================================================

conn.close()

print("\n=================================================")
print("        END OF VIEWS MASTER DEMO")
print("=================================================")

           SQL VIEWS MASTER DEMO

Tables Created!
Sample Data Inserted!

Simple View Result:
[('Rahul', 'Hyderabad'), ('Anjali', 'Bangalore'), ('Kiran', 'Hyderabad')]

View with WHERE Result:
[(1, 'Rahul', 'Hyderabad'), (3, 'Kiran', 'Hyderabad')]

View with JOIN Result:
[('Rahul', 5000), ('Rahul', 3000), ('Anjali', 7000), ('Kiran', 2000)]

View with Aggregation Result:
[('Anjali', 7000), ('Kiran', 2000), ('Rahul', 8000)]

Updating Base Table (View Reflects Change)
[('Rahul', 'Hyderabad'), ('Anjali', 'Bangalore'), ('Kiran', 'Chennai')]

List of Views in Database:
[('view_customers',), ('view_hyderabad_customers',), ('view_customer_orders',), ('view_total_orders',)]

Dropped view_hyderabad_customers

        END OF VIEWS MASTER DEMO


BEFORE triggers
✔ AFTER triggers
✔ INSERT / UPDATE / DELETE triggers
✔ Validation triggers
✔ Audit logging
✔ Business rule enforcement

In [7]:
"""
=========================================================
            COMPLETE SQL TRIGGERS MASTER DEMO
=========================================================
This script demonstrates:

✔ What is a Trigger
✔ BEFORE INSERT Trigger
✔ AFTER INSERT Trigger
✔ AFTER UPDATE Trigger
✔ AFTER DELETE Trigger
✔ Audit Logging
✔ Business Rule Validation
✔ Preventing Invalid Data
✔ Real-world Examples
=========================================================
"""

import sqlite3

print("=================================================")
print("           SQL TRIGGERS MASTER DEMO")
print("=================================================")

# =================================================
# 1️⃣ Connect Database
# =================================================

conn = sqlite3.connect("triggers_demo.db")
cursor = conn.cursor()

cursor.execute("PRAGMA foreign_keys = ON;")

# =================================================
# 2️⃣ Create Base Tables
# =================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    salary INTEGER NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS audit_log (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    action TEXT,
    emp_name TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

conn.commit()
print("\nTables Created!")

# =================================================
# 3️⃣ AFTER INSERT Trigger (Audit Log)
# =================================================

cursor.execute("""
CREATE TRIGGER IF NOT EXISTS after_employee_insert
AFTER INSERT ON employees
BEGIN
    INSERT INTO audit_log(action, emp_name)
    VALUES ('INSERT', NEW.name);
END;
""")

# =================================================
# 4️⃣ AFTER UPDATE Trigger
# =================================================

cursor.execute("""
CREATE TRIGGER IF NOT EXISTS after_employee_update
AFTER UPDATE ON employees
BEGIN
    INSERT INTO audit_log(action, emp_name)
    VALUES ('UPDATE', NEW.name);
END;
""")

# =================================================
# 5️⃣ AFTER DELETE Trigger
# =================================================

cursor.execute("""
CREATE TRIGGER IF NOT EXISTS after_employee_delete
AFTER DELETE ON employees
BEGIN
    INSERT INTO audit_log(action, emp_name)
    VALUES ('DELETE', OLD.name);
END;
""")

# =================================================
# 6️⃣ BEFORE INSERT Trigger (Validation)
# Prevent salary < 30000
# =================================================

cursor.execute("""
CREATE TRIGGER IF NOT EXISTS validate_salary
BEFORE INSERT ON employees
WHEN NEW.salary < 30000
BEGIN
    SELECT RAISE(ABORT, 'Salary must be >= 30000');
END;
""")

conn.commit()
print("Triggers Created Successfully!")

# =================================================
# 7️⃣ Test INSERT (Valid)
# =================================================

print("\nInserting Valid Employee...")
cursor.execute("INSERT INTO employees (name, salary) VALUES (?, ?)",
               ("Rahul", 50000))
conn.commit()

# =================================================
# 8️⃣ Test INSERT (Invalid Salary)
# =================================================

try:
    print("\nInserting Invalid Employee...")
    cursor.execute("INSERT INTO employees (name, salary) VALUES (?, ?)",
                   ("LowSalary", 20000))
    conn.commit()
except Exception as e:
    print("Trigger Blocked Insert:", e)

# =================================================
# 9️⃣ Test UPDATE
# =================================================

cursor.execute("UPDATE employees SET salary=60000 WHERE name='Rahul'")
conn.commit()

# =================================================
# 🔟 Test DELETE
# =================================================

cursor.execute("DELETE FROM employees WHERE name='Rahul'")
conn.commit()

# =================================================
# 1️⃣1️⃣ View Audit Logs
# =================================================

print("\nAudit Log Records:")
cursor.execute("SELECT * FROM audit_log")
print(cursor.fetchall())

# =================================================
# 1️⃣2️⃣ View Existing Triggers
# =================================================

print("\nList of Triggers:")
cursor.execute("""
SELECT name FROM sqlite_master
WHERE type='trigger';
""")
print(cursor.fetchall())

# =================================================
# Close Connection
# =================================================

conn.close()

print("\n=================================================")
print("        END OF TRIGGERS MASTER DEMO")
print("=================================================")

           SQL TRIGGERS MASTER DEMO

Tables Created!
Triggers Created Successfully!

Inserting Valid Employee...

Inserting Invalid Employee...
Trigger Blocked Insert: Salary must be >= 30000

Audit Log Records:
[(1, 'INSERT', 'Rahul', '2026-02-20 13:24:24'), (2, 'UPDATE', 'Rahul', '2026-02-20 13:24:24'), (3, 'DELETE', 'Rahul', '2026-02-20 13:24:24')]

List of Triggers:
[('after_employee_insert',), ('after_employee_update',), ('after_employee_delete',), ('validate_salary',)]

        END OF TRIGGERS MASTER DEMO


In [8]:
"""
=========================================================
        STORED-LIKE LOGIC MASTER DEMO (SQLite)
=========================================================
Simulates Stored Procedures using:

✔ Python functions as procedures
✔ Transactions
✔ Validation logic
✔ Business rules
✔ Triggers
✔ Atomic operations
✔ Reporting queries
=========================================================
"""

import sqlite3

print("=================================================")
print("       STORED-LIKE LOGIC MASTER DEMO")
print("=================================================")

# =================================================
# 1️⃣ Connect Database
# =================================================

conn = sqlite3.connect("stored_logic_demo.db")
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

# =================================================
# 2️⃣ Create Tables
# =================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS accounts (
    account_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    balance INTEGER NOT NULL CHECK(balance >= 0)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS transactions (
    txn_id INTEGER PRIMARY KEY AUTOINCREMENT,
    from_account INTEGER,
    to_account INTEGER,
    amount INTEGER,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

conn.commit()
print("Tables Created!")

# =================================================
# 3️⃣ Insert Sample Accounts
# =================================================

cursor.execute("INSERT INTO accounts (name, balance) VALUES ('Rahul', 10000)")
cursor.execute("INSERT INTO accounts (name, balance) VALUES ('Anjali', 5000)")
conn.commit()

print("Sample Accounts Inserted!")

# =================================================
# 4️⃣ Stored-Like Procedure: Transfer Money
# =================================================

def transfer_money(from_id, to_id, amount):
    try:
        conn.execute("BEGIN")

        # Check balance
        cursor.execute("SELECT balance FROM accounts WHERE account_id=?", (from_id,))
        result = cursor.fetchone()

        if result is None:
            raise Exception("From account not found")

        if result[0] < amount:
            raise Exception("Insufficient balance")

        # Deduct from sender
        cursor.execute("""
        UPDATE accounts
        SET balance = balance - ?
        WHERE account_id=?
        """, (amount, from_id))

        # Add to receiver
        cursor.execute("""
        UPDATE accounts
        SET balance = balance + ?
        WHERE account_id=?
        """, (amount, to_id))

        # Log transaction
        cursor.execute("""
        INSERT INTO transactions (from_account, to_account, amount)
        VALUES (?, ?, ?)
        """, (from_id, to_id, amount))

        conn.commit()
        print(f"Transfer of {amount} successful!")

    except Exception as e:
        conn.rollback()
        print("Transaction Failed:", e)

# =================================================
# 5️⃣ Execute Stored-Like Procedure
# =================================================

print("\nExecuting Transfer Procedure...")
transfer_money(1, 2, 2000)

# =================================================
# 6️⃣ View Updated Balances
# =================================================

print("\nAccount Balances:")
cursor.execute("SELECT * FROM accounts")
print(cursor.fetchall())

# =================================================
# 7️⃣ Reporting Procedure (Aggregation)
# =================================================

def get_total_transactions():
    cursor.execute("SELECT SUM(amount) FROM transactions")
    total = cursor.fetchone()[0]
    return total if total else 0

print("\nTotal Transaction Amount:", get_total_transactions())

# =================================================
# 8️⃣ Business Rule Validation (Procedure-like)
# =================================================

def create_account(name, balance):
    if balance < 1000:
        print("Minimum opening balance is 1000")
        return

    cursor.execute("""
    INSERT INTO accounts (name, balance)
    VALUES (?, ?)
    """, (name, balance))
    conn.commit()
    print("Account Created!")

print("\nCreating New Account...")
create_account("Kiran", 800)   # Should fail
create_account("Kiran", 3000)  # Should succeed

# =================================================
# 9️⃣ Trigger for Automatic Logging
# =================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS audit_log (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    message TEXT,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

cursor.execute("""
CREATE TRIGGER IF NOT EXISTS log_account_insert
AFTER INSERT ON accounts
BEGIN
    INSERT INTO audit_log(message)
    VALUES ('New account created');
END;
""")

conn.commit()

cursor.execute("INSERT INTO accounts (name, balance) VALUES ('TestUser', 4000)")
conn.commit()

print("\nAudit Logs:")
cursor.execute("SELECT * FROM audit_log")
print(cursor.fetchall())

# =================================================
# 🔟 View Transactions
# =================================================

print("\nTransaction Records:")
cursor.execute("SELECT * FROM transactions")
print(cursor.fetchall())

# =================================================
# Close Database
# =================================================

conn.close()

print("\n=================================================")
print("        END OF STORED-LIKE LOGIC DEMO")
print("=================================================")

       STORED-LIKE LOGIC MASTER DEMO
Tables Created!
Sample Accounts Inserted!

Executing Transfer Procedure...
Transfer of 2000 successful!

Account Balances:
[(1, 'Rahul', 8000), (2, 'Anjali', 7000)]

Total Transaction Amount: 2000

Creating New Account...
Minimum opening balance is 1000
Account Created!

Audit Logs:
[(1, 'New account created', '2026-02-20 13:27:00')]

Transaction Records:
[(1, 1, 2, 2000, '2026-02-20 13:27:00')]

        END OF STORED-LIKE LOGIC DEMO


In [9]:
"""
=========================================================
 TRANSACTIONS + JOINS + SUBQUERIES MASTER DEMO (SQLite)
=========================================================

This script demonstrates:

✔ Transactions (BEGIN, COMMIT, ROLLBACK)
✔ Atomic operations
✔ INNER JOIN
✔ LEFT JOIN
✔ CROSS JOIN
✔ SELF JOIN
✔ Subqueries
✔ Correlated Subqueries
✔ EXISTS
✔ IN
✔ Nested Subqueries
✔ Real-world examples
=========================================================
"""

import sqlite3

print("=================================================")
print(" TRANSACTIONS + JOINS + SUBQUERIES MASTER DEMO")
print("=================================================")

# =================================================
# 1️⃣ Connect Database
# =================================================

conn = sqlite3.connect("master_demo.db")
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

# =================================================
# 2️⃣ Create Tables
# =================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    city TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    amount INTEGER,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
""")

conn.commit()
print("Tables Created!")

# =================================================
# 3️⃣ Insert Sample Data
# =================================================

cursor.executemany("""
INSERT INTO customers (name, city)
VALUES (?, ?)
""", [
    ("Rahul", "Hyderabad"),
    ("Anjali", "Bangalore"),
    ("Kiran", "Chennai")
])

cursor.executemany("""
INSERT INTO orders (customer_id, amount)
VALUES (?, ?)
""", [
    (1, 5000),
    (1, 3000),
    (2, 7000),
    (3, 2000)
])

conn.commit()
print("Sample Data Inserted!")

# =================================================
# 4️⃣ TRANSACTIONS DEMO
# =================================================

print("\n--- TRANSACTION DEMO ---")

try:
    conn.execute("BEGIN")

    # Deduct amount (simulate payment)
    cursor.execute("UPDATE orders SET amount = amount - 1000 WHERE order_id = 1")

    # Force error (uncomment to test rollback)
    # raise Exception("Forced Error")

    conn.commit()
    print("Transaction Successful!")

except Exception as e:
    conn.rollback()
    print("Transaction Failed:", e)

# =================================================
# 5️⃣ JOINS
# =================================================

print("\n--- INNER JOIN ---")
cursor.execute("""
SELECT c.name, o.amount
FROM customers c
INNER JOIN orders o
ON c.customer_id = o.customer_id
""")
print(cursor.fetchall())

print("\n--- LEFT JOIN ---")
cursor.execute("""
SELECT c.name, o.amount
FROM customers c
LEFT JOIN orders o
ON c.customer_id = o.customer_id
""")
print(cursor.fetchall())

print("\n--- CROSS JOIN ---")
cursor.execute("""
SELECT c.name, o.amount
FROM customers c
CROSS JOIN orders o
LIMIT 5
""")
print(cursor.fetchall())

# SELF JOIN (Example: Customer referring another customer)
cursor.execute("""
CREATE TABLE IF NOT EXISTS referrals (
    customer_id INTEGER,
    referred_by INTEGER,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY(referred_by) REFERENCES customers(customer_id)
);
""")

cursor.execute("INSERT INTO referrals VALUES (2,1)")
conn.commit()

print("\n--- SELF JOIN ---")
cursor.execute("""
SELECT c1.name AS Customer, c2.name AS Referred_By
FROM referrals r
JOIN customers c1 ON r.customer_id = c1.customer_id
JOIN customers c2 ON r.referred_by = c2.customer_id
""")
print(cursor.fetchall())

# =================================================
# 6️⃣ SUBQUERIES
# =================================================

print("\n--- Subquery (IN) ---")
cursor.execute("""
SELECT name FROM customers
WHERE customer_id IN (
    SELECT customer_id FROM orders WHERE amount > 4000
)
""")
print(cursor.fetchall())

print("\n--- Subquery (EXISTS) ---")
cursor.execute("""
SELECT name FROM customers c
WHERE EXISTS (
    SELECT 1 FROM orders o
    WHERE o.customer_id = c.customer_id
)
""")
print(cursor.fetchall())

print("\n--- Correlated Subquery ---")
cursor.execute("""
SELECT name,
    (SELECT SUM(amount)
     FROM orders o
     WHERE o.customer_id = c.customer_id) AS total_spent
FROM customers c
""")
print(cursor.fetchall())

print("\n--- Nested Subquery ---")
cursor.execute("""
SELECT name FROM customers
WHERE customer_id = (
    SELECT customer_id FROM orders
    WHERE amount = (
        SELECT MAX(amount) FROM orders
    )
)
""")
print(cursor.fetchall())

# =================================================
# 7️⃣ Aggregation with JOIN
# =================================================

print("\n--- Aggregation with JOIN ---")
cursor.execute("""
SELECT c.name, SUM(o.amount) as total
FROM customers c
JOIN orders o
ON c.customer_id = o.customer_id
GROUP BY c.name
""")
print(cursor.fetchall())

# =================================================
# Close Database
# =================================================

conn.close()

print("\n=================================================")
print("        END OF MASTER DEMO")
print("=================================================")

 TRANSACTIONS + JOINS + SUBQUERIES MASTER DEMO
Tables Created!
Sample Data Inserted!

--- TRANSACTION DEMO ---
Transaction Successful!

--- INNER JOIN ---
[('Rahul', 4000), ('Rahul', 3000), ('Anjali', 7000), ('Kiran', 2000)]

--- LEFT JOIN ---
[('Rahul', 3000), ('Rahul', 4000), ('Anjali', 7000), ('Kiran', 2000)]

--- CROSS JOIN ---
[('Rahul', 4000), ('Rahul', 3000), ('Rahul', 7000), ('Rahul', 2000), ('Anjali', 4000)]

--- SELF JOIN ---
[('Anjali', 'Rahul')]

--- Subquery (IN) ---
[('Anjali',)]

--- Subquery (EXISTS) ---
[('Rahul',), ('Anjali',), ('Kiran',)]

--- Correlated Subquery ---
[('Rahul', 7000), ('Anjali', 7000), ('Kiran', 2000)]

--- Nested Subquery ---
[('Anjali',)]

--- Aggregation with JOIN ---
[('Anjali', 7000), ('Kiran', 2000), ('Rahul', 7000)]

        END OF MASTER DEMO


In [10]:
"""
=========================================================
 SUBQUERIES + AGGREGATIONS + PERFORMANCE MASTER DEMO
=========================================================

This script demonstrates:

✔ Subqueries (IN, EXISTS, Correlated, Nested)
✔ Aggregation functions (SUM, AVG, COUNT, MAX, MIN)
✔ GROUP BY & HAVING
✔ CASE with Aggregation
✔ Performance Queries
✔ EXPLAIN QUERY PLAN
✔ Index impact on performance
✔ Real-world reporting queries
=========================================================
"""

import sqlite3
import random

print("=================================================")
print(" SUBQUERIES + AGGREGATIONS + PERFORMANCE DEMO")
print("=================================================")

# =================================================
# 1️⃣ Connect Database
# =================================================

conn = sqlite3.connect("performance_demo.db")
cursor = conn.cursor()

# =================================================
# 2️⃣ Create Tables
# =================================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    department TEXT,
    salary INTEGER
);
""")

conn.commit()

# =================================================
# 3️⃣ Insert Large Data (Performance Testing)
# =================================================

departments = ["IT", "HR", "Finance", "Marketing"]

for i in range(3000):
    cursor.execute("""
    INSERT INTO employees (name, department, salary)
    VALUES (?, ?, ?)
    """, (
        f"Emp{i}",
        random.choice(departments),
        random.randint(30000, 150000)
    ))

conn.commit()
print("3000 Records Inserted!")

# =================================================
# 4️⃣ AGGREGATIONS
# =================================================

print("\n--- Basic Aggregations ---")

cursor.execute("SELECT COUNT(*) FROM employees")
print("Total Employees:", cursor.fetchone())

cursor.execute("SELECT AVG(salary) FROM employees")
print("Average Salary:", cursor.fetchone())

cursor.execute("SELECT MAX(salary) FROM employees")
print("Max Salary:", cursor.fetchone())

cursor.execute("SELECT MIN(salary) FROM employees")
print("Min Salary:", cursor.fetchone())

# GROUP BY
print("\n--- GROUP BY Department ---")
cursor.execute("""
SELECT department, COUNT(*), AVG(salary)
FROM employees
GROUP BY department
""")
print(cursor.fetchall())

# HAVING
print("\n--- HAVING (Departments with Avg Salary > 80000) ---")
cursor.execute("""
SELECT department, AVG(salary) as avg_sal
FROM employees
GROUP BY department
HAVING avg_sal > 80000
""")
print(cursor.fetchall())

# =================================================
# 5️⃣ SUBQUERIES
# =================================================

print("\n--- Subquery (IN) ---")
cursor.execute("""
SELECT name FROM employees
WHERE salary IN (
    SELECT MAX(salary) FROM employees
)
""")
print(cursor.fetchall())

print("\n--- Subquery (EXISTS) ---")
cursor.execute("""
SELECT name FROM employees e
WHERE EXISTS (
    SELECT 1 FROM employees
    WHERE department = e.department
    AND salary > 100000
)
LIMIT 5
""")
print(cursor.fetchall())

print("\n--- Correlated Subquery ---")
cursor.execute("""
SELECT name,
    (SELECT AVG(salary)
     FROM employees e2
     WHERE e2.department = e1.department)
FROM employees e1
LIMIT 5
""")
print(cursor.fetchall())

print("\n--- Nested Subquery ---")
cursor.execute("""
SELECT name FROM employees
WHERE salary = (
    SELECT MAX(salary)
    FROM employees
    WHERE department = (
        SELECT department FROM employees
        ORDER BY salary DESC
        LIMIT 1
    )
)
""")
print(cursor.fetchall())

# =================================================
# 6️⃣ CASE with Aggregation
# =================================================

print("\n--- Salary Category (CASE) ---")
cursor.execute("""
SELECT department,
    SUM(CASE WHEN salary > 100000 THEN 1 ELSE 0 END) as high_salary_count
FROM employees
GROUP BY department
""")
print(cursor.fetchall())

# =================================================
# 7️⃣ PERFORMANCE ANALYSIS
# =================================================

print("\n--- Query Plan WITHOUT Index ---")
cursor.execute("""
EXPLAIN QUERY PLAN
SELECT * FROM employees WHERE department='IT'
""")
print(cursor.fetchall())

# Create Index
cursor.execute("CREATE INDEX idx_department ON employees(department)")
conn.commit()

print("\nIndex Created on department!")

print("\n--- Query Plan WITH Index ---")
cursor.execute("""
EXPLAIN QUERY PLAN
SELECT * FROM employees WHERE department='IT'
""")
print(cursor.fetchall())

# =================================================
# 8️⃣ Optimized Query Example
# =================================================

print("\n--- Optimized Query with LIMIT ---")
cursor.execute("""
SELECT name, salary
FROM employees
WHERE department='IT'
ORDER BY salary DESC
LIMIT 5
""")
print(cursor.fetchall())

# =================================================
# Close Database
# =================================================

conn.close()

print("\n=================================================")
print("        END OF MASTER DEMO")
print("=================================================")

 SUBQUERIES + AGGREGATIONS + PERFORMANCE DEMO
3000 Records Inserted!

--- Basic Aggregations ---
Total Employees: (3000,)
Average Salary: (90034.06,)
Max Salary: (150000,)
Min Salary: (30013,)

--- GROUP BY Department ---
[('Finance', 745, 87805.64697986578), ('HR', 732, 90321.96721311475), ('IT', 739, 90391.53585926928), ('Marketing', 784, 91545.85204081633)]

--- HAVING (Departments with Avg Salary > 80000) ---
[('Finance', 87805.64697986578), ('HR', 90321.96721311475), ('IT', 90391.53585926928), ('Marketing', 91545.85204081633)]

--- Subquery (IN) ---
[('Emp563',)]

--- Subquery (EXISTS) ---
[('Emp0',), ('Emp1',), ('Emp2',), ('Emp3',), ('Emp4',)]

--- Correlated Subquery ---
[('Emp0', 90391.53585926928), ('Emp1', 90391.53585926928), ('Emp2', 90321.96721311475), ('Emp3', 91545.85204081633), ('Emp4', 87805.64697986578)]

--- Nested Subquery ---
[('Emp563',)]

--- Salary Category (CASE) ---
[('Finance', 292), ('HR', 305), ('IT', 314), ('Marketing', 349)]

--- Query Plan WITHOUT Index -

In [11]:
"""
=========================================================
        REAL-WORLD JOB PORTAL SCHEMA (PRODUCTION)
=========================================================

This script demonstrates:

✔ Proper table design
✔ Relationships (1-1, 1-M, M-M)
✔ Constraints
✔ Indexing
✔ Views
✔ Reporting queries
✔ Transactions
✔ Cleanup section
=========================================================
"""

import sqlite3

print("=================================================")
print("      REAL-WORLD JOB PORTAL SCHEMA DEMO")
print("=================================================")

conn = sqlite3.connect("job_portal_real.db")
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

# =================================================
# 1️⃣ LOOKUP TABLES
# =================================================

cursor.execute("""
CREATE TABLE roles (
    role_id INTEGER PRIMARY KEY AUTOINCREMENT,
    role_name TEXT UNIQUE NOT NULL
);
""")

cursor.execute("""
CREATE TABLE job_types (
    job_type_id INTEGER PRIMARY KEY AUTOINCREMENT,
    type_name TEXT UNIQUE NOT NULL
);
""")

# =================================================
# 2️⃣ USERS TABLE
# =================================================

cursor.execute("""
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    role_id INTEGER NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(role_id) REFERENCES roles(role_id)
);
""")

# =================================================
# 3️⃣ COMPANIES TABLE
# =================================================

cursor.execute("""
CREATE TABLE companies (
    company_id INTEGER PRIMARY KEY AUTOINCREMENT,
    company_name TEXT UNIQUE NOT NULL,
    location TEXT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

# =================================================
# 4️⃣ JOBS TABLE
# =================================================

cursor.execute("""
CREATE TABLE jobs (
    job_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT NOT NULL,
    salary INTEGER CHECK(salary > 0),
    company_id INTEGER,
    job_type_id INTEGER,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(company_id) REFERENCES companies(company_id)
        ON DELETE CASCADE,
    FOREIGN KEY(job_type_id) REFERENCES job_types(job_type_id)
);
""")

# =================================================
# 5️⃣ APPLICATIONS (Many-to-Many)
# =================================================

cursor.execute("""
CREATE TABLE applications (
    user_id INTEGER,
    job_id INTEGER,
    status TEXT DEFAULT 'Applied',
    applied_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY(user_id, job_id),
    FOREIGN KEY(user_id) REFERENCES users(user_id)
        ON DELETE CASCADE,
    FOREIGN KEY(job_id) REFERENCES jobs(job_id)
        ON DELETE CASCADE
);
""")

# =================================================
# 6️⃣ SKILLS + USER_SKILLS (Many-to-Many)
# =================================================

cursor.execute("""
CREATE TABLE skills (
    skill_id INTEGER PRIMARY KEY AUTOINCREMENT,
    skill_name TEXT UNIQUE NOT NULL
);
""")

cursor.execute("""
CREATE TABLE user_skills (
    user_id INTEGER,
    skill_id INTEGER,
    PRIMARY KEY(user_id, skill_id),
    FOREIGN KEY(user_id) REFERENCES users(user_id)
        ON DELETE CASCADE,
    FOREIGN KEY(skill_id) REFERENCES skills(skill_id)
        ON DELETE CASCADE
);
""")

conn.commit()
print("Tables Created Successfully!")

# =================================================
# 7️⃣ INDEXING FOR PERFORMANCE
# =================================================

cursor.execute("CREATE INDEX idx_users_email ON users(email);")
cursor.execute("CREATE INDEX idx_jobs_salary ON jobs(salary);")
cursor.execute("CREATE INDEX idx_applications_status ON applications(status);")
conn.commit()

print("Indexes Created!")

# =================================================
# 8️⃣ INSERT SAMPLE DATA
# =================================================

cursor.executemany("INSERT INTO roles(role_name) VALUES (?)",
                   [("Admin",), ("Recruiter",), ("Candidate",)])

cursor.executemany("INSERT INTO job_types(type_name) VALUES (?)",
                   [("Full-Time",), ("Part-Time",), ("Internship",)])

cursor.execute("""
INSERT INTO users(first_name, last_name, email, password_hash, role_id)
VALUES ('Rahul', 'Kumar', 'rahul@gmail.com', 'hash123', 3)
""")

cursor.execute("""
INSERT INTO companies(company_name, location)
VALUES ('TechCorp', 'Hyderabad')
""")

cursor.execute("""
INSERT INTO jobs(title, description, salary, company_id, job_type_id)
VALUES ('Backend Developer', 'API development', 800000, 1, 1)
""")

cursor.execute("""
INSERT INTO applications(user_id, job_id)
VALUES (1,1)
""")

conn.commit()
print("Sample Data Inserted!")

# =================================================
# 9️⃣ VIEW (Reporting Layer)
# =================================================

cursor.execute("""
CREATE VIEW job_application_report AS
SELECT u.first_name || ' ' || u.last_name AS candidate,
       j.title,
       c.company_name,
       a.status
FROM applications a
JOIN users u ON a.user_id = u.user_id
JOIN jobs j ON a.job_id = j.job_id
JOIN companies c ON j.company_id = c.company_id;
""")

print("\nJob Application Report:")
cursor.execute("SELECT * FROM job_application_report")
print(cursor.fetchall())

# =================================================
# 🔟 AGGREGATION REPORT
# =================================================

print("\nApplications Count Per Job:")
cursor.execute("""
SELECT j.title, COUNT(*) as total_applications
FROM applications a
JOIN jobs j ON a.job_id = j.job_id
GROUP BY j.title
""")
print(cursor.fetchall())

# =================================================
# 1️⃣1️⃣ TRANSACTION EXAMPLE
# =================================================

try:
    conn.execute("BEGIN")
    cursor.execute("""
    INSERT INTO jobs(title, description, salary, company_id, job_type_id)
    VALUES ('Frontend Developer', 'React developer', 700000, 1, 1)
    """)
    conn.commit()
    print("\nTransaction Successful!")
except:
    conn.rollback()
    print("Transaction Failed!")

# =================================================
# CLEANUP SECTION
# =================================================

print("\n--- CLEANUP SECTION ---")

def cleanup_database():
    cursor.execute("DROP VIEW IF EXISTS job_application_report")
    cursor.execute("DROP TABLE IF EXISTS user_skills")
    cursor.execute("DROP TABLE IF EXISTS skills")
    cursor.execute("DROP TABLE IF EXISTS applications")
    cursor.execute("DROP TABLE IF EXISTS jobs")
    cursor.execute("DROP TABLE IF EXISTS companies")
    cursor.execute("DROP TABLE IF EXISTS users")
    cursor.execute("DROP TABLE IF EXISTS roles")
    cursor.execute("DROP TABLE IF EXISTS job_types")
    conn.commit()
    print("All Tables & Views Dropped Successfully!")

# Uncomment below to cleanup
# cleanup_database()

# =================================================
# CLOSE CONNECTION
# =================================================

conn.close()

print("\n=================================================")
print("        END OF JOB PORTAL SCHEMA DEMO")
print("=================================================")

      REAL-WORLD JOB PORTAL SCHEMA DEMO
Tables Created Successfully!
Indexes Created!
Sample Data Inserted!

Job Application Report:
[('Rahul Kumar', 'Backend Developer', 'TechCorp', 'Applied')]

Applications Count Per Job:
[('Backend Developer', 1)]

Transaction Successful!

--- CLEANUP SECTION ---

        END OF JOB PORTAL SCHEMA DEMO
